Matrix Factorization

In [38]:
# imports

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

from torch.utils.data import TensorDataset, DataLoader

Load Data

In [6]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip


--2023-01-27 12:03:19--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.1’

ml-100k.zip.1       100%[===================>]   4.70M  14.3MB/s    in 0.3s    

2023-01-27 12:03:20 (14.3 MB/s) - ‘ml-100k.zip.1’ saved [4924029/4924029]



In [8]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [3]:
!rm ml-100k.zip
%cd ml-100k

rm: ml-100k.zip: No such file or directory
[Errno 2] No such file or directory: 'ml-100k'
/Users/darylfung/programming/pytorch-recommender-system/notebooks/ml-100k


In [84]:
# read csv file
data = pd.read_csv('u.data', '\t', names=['user', 'item', 'rating', 'timestamp'])

In [5]:
data.head

<bound method NDFrame.head of        user  item  rating  timestamp
0       196   242       3  881250949
1       186   302       3  891717742
2        22   377       1  878887116
3       244    51       2  880606923
4       166   346       1  886397596
...     ...   ...     ...        ...
99995   880   476       3  880175444
99996   716   204       5  879795543
99997   276  1090       1  874795795
99998    13   225       2  882399156
99999    12   203       3  879959583

[100000 rows x 4 columns]>

Create network

In [72]:
class Embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_dims):
        super(Embedding, self).__init__()
        self.user_matrix = nn.Embedding(num_users, embedding_dims)
        self.item_matrix = nn.Embedding(num_items, embedding_dims)

    def forward(self, user, item):
        if type(user) != torch.Tensor:
            user = torch.tensor(user)
        if type(item) != torch.Tensor:
            item = torch.tensor(item)

        user_matrix = self.user_matrix(user)
        item_matrix = self.item_matrix(item)
        outputs = user_matrix @ item_matrix.T
        return outputs

196

Test network

In [85]:
num_users = data.max()['user']+1
num_items = data.max()['item']+1

network = Embedding(int(num_users), int(num_items), 64)
predicted_rating = network(data['user'][0], data['item'][0])

Create loss functon and optimizer

In [86]:
criterion = nn.MSELoss()
optim = torch.optim.Adam(network.parameters(), lr=0.001)

In [42]:
# rating_scale = StandardScaler()
# ratings = np.expand_dims(data['rating'].values, 1)
# normalize_rating = rating_scale.fit_transform(ratings)
# data['rating'] = normalize_rating.reshape(-1)

In [87]:
dataset = TensorDataset(torch.FloatTensor(data.values))
data_loader = DataLoader(dataset, 128, shuffle=False)

for i in range(5):
    for train_x in data_loader:
        optim.zero_grad()
        user = train_x[0][:, 0].long()
        item = train_x[0][:, 1].long()
        ratings = train_x[0][:, 2]
        predicted_rating = network(user, item)

        loss = criterion(predicted_rating, ratings)
        loss.backward()
        optim.step()
    print(loss)

/Users/darylfung/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/darylfung/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(39.3559, grad_fn=<MseLossBackward>)
tensor(23.3093, grad_fn=<MseLossBackward>)
tensor(12.5781, grad_fn=<MseLossBackward>)
tensor(7.4742, grad_fn=<MseLossBackward>)
tensor(4.8121, grad_fn=<MseLossBackward>)


In [88]:
predicted_rating = network(data['user'][1], data['user'][5])